In [18]:
import requests
import pandas as pd
import os
import json
import datetime
from datetime import datetime, timedelta, date
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import numpy as np
load_dotenv()

import warnings
warnings.filterwarnings("ignore")

In [88]:
#function to pull data from systradingmarketdata

def get_marketdata(start_datetime, end_datetime, analysis_group):
    #define the basic parameters of the API call
    baseurl = "https://systradingmarketdataapi.azurewebsites.net/api/"
    url = f"{baseurl}Authentication/request"
    
    payload=f'{{"username": "{os.getenv("MARKETDATA_USER")}", "password": "{os.getenv("MARKETDATA_PWD")}"}}'
    headers = {
      'Authorization': '',
      'Content-Type': 'application/json',
    }
    
    # get the token
    login_response = requests.request("POST", url, headers=headers, data=payload)
    
    params = f"start={start_datetime}&end={end_datetime}&granularity=hours&timeZone=GMT"
    #analysis_group = "1Base_EC%20Generation%20by%20Fuel%20Type"
    group_url = f'{baseurl}AnalysisGroup/AllCurves/{analysis_group}?{params}'

    payload={}
    headers = {
      'Authorization': f'Bearer {login_response.text}',
    }

    response = requests.request("GET", group_url, headers=headers, data=payload)

    #print out the json
    #print(response.text)
    # %%

    power_json = json.loads(response.text)

    # loading hour sequence from the maximum and minimum found - it's not yet clear
    # the logic for the cut off date times.
    max_found_time = max([x['timeSeries'][-1]['date'] for x in power_json['curves']])
    min_found_time = min([x['timeSeries'][0]['date'] for x in power_json['curves']])

    hour_sequence = pd.date_range(start = pd.to_datetime(min_found_time),
                             end = pd.to_datetime(max_found_time),
                             freq = "H")
    # %%
    # create named dataframe from json
    def turn_series_to_df(json_series):
        this_df = pd.DataFrame.from_records(json_series['timeSeries'])
        this_df['date'] = pd.to_datetime(this_df['date'])
        this_df.set_index('date', inplace=True)
        this_df.rename(columns={'value' : json_series['memberName']}, inplace=True)#'name' is an alternative here to memberName
        return(this_df)

    # %%
    # load initial dataset for first time series and fill mising hours

    power_df = turn_series_to_df(power_json['curves'][0])
    power_df = power_df.reindex(hour_sequence)
    power_df.rename_axis("date", axis='index', inplace=True)

    # %%

    for full_series in power_json['curves'][1:]:
        this_df = turn_series_to_df(full_series)
        power_df = pd.merge(power_df,this_df,how='left', left_index=True, right_index=True)

    return power_df

In [89]:
#Pull the data and produce the forecast

dt14 = datetime.today() + timedelta(days = 15)
dt14=datetime(dt14.year,dt14.month,dt14.day)

ce = get_marketdata(datetime(2021,10,1), dt14, 'weather_CEE')


ce = ce.resample(rule='24H', closed='left', label='left', base=5).mean().round(1)
ce['date'] = ce.apply(lambda x: datetime(x.name.year, x.name.month, x.name.day), axis = 1)
ce = ce.set_index('date')
ce = ce.rename(columns = {ce.columns[0]:'obs_Kyiv', ce.columns[1]:'fcst_Kyiv'})
#ce = ce.set_index['date']

comp = pd.DataFrame(index =pd.date_range(start = datetime.today(), end = dt14+ timedelta(days =-1)), columns = ['FCST'])
comp = comp.resample('D').last()
comp['FCST'] = comp.apply(lambda x: ce['fcst_Kyiv'][ce.index==x.name].iloc[0], axis = 1) # if ((x.name >= datetime.today()) & ((x.name - datetime.today()).days<14)) else np.nan

coefs = pd.read_csv('coef.csv')
slope = coefs['slope'].iloc[0]
inter = coefs['intercept'].iloc[0]

comp['DEM_FCST'] = comp['FCST'] * slope  + inter + 3.75

In [107]:
#save the output in the folder
#!!! This part needs to be re-written as the output should be saved in the database
comp.to_csv('output/forecast_'+datetime.today().strftime("%Y-%m-%d")+'.csv')

In [117]:
#compare with the previous forecast

#!!! this part should be re-written as we should pull the previous forecast from the database
prevdate = datetime.now()+timedelta(days = -3) if datetime.today().weekday() == 0 else datetime.now()+timedelta(days = -1)
prevforecast = pd.read_csv('output/forecast_'+prevdate.strftime("%Y-%m-%d")+'.csv', index_col =0)
prevforecast.index = pd.to_datetime(prevforecast.index)
#prevdate = datetime(prevdate.year, prevdate.month, prevdate.day)

In [122]:
#calculate day-on-day change
dod = pd.DataFrame((prevforecast['DEM_FCST']-comp['DEM_FCST']).dropna())

In [128]:
#Print the results (we should add this to our daily emails)
print('Day-on-day change:')
print(dod.round())

print('Current forecast:')
print(comp['DEM_FCST'].round(1))

Day-on-day change:
            DEM_FCST
2021-12-20      -1.0
2021-12-21      -8.0
2021-12-22      -9.0
2021-12-23      -9.0
2021-12-24      -5.0
2021-12-25      -2.0
2021-12-26      -1.0
2021-12-27      -0.0
2021-12-28       0.0
2021-12-29      -1.0
2021-12-30      -2.0
Current forecast:
2021-12-20    116.0
2021-12-21    122.5
2021-12-22    124.4
2021-12-23    123.6
2021-12-24    119.9
2021-12-25    117.5
2021-12-26    116.4
2021-12-27    115.2
2021-12-28    114.8
2021-12-29    115.6
2021-12-30    117.1
2021-12-31    118.3
2022-01-01    119.5
2022-01-02    120.1
Freq: D, Name: DEM_FCST, dtype: float64
